In [1]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [2]:
QPATH = "Quantlet/4-qode2desc"

In [3]:
import sys

IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    %load_ext lab_black

In [4]:
import pickle
import json
import re
import sys
from IPython.display import display
import datetime

from tqdm import tqdm
import pandas as pd
import numpy as np

tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"

import nltk

nltk.download("punkt")
import evaluate

from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import importlib
import analysis_modules

importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/4-qode2desc/analysis_modules.py'>

In [6]:
# model_name = '../5-domain-pre-training/analysis_report_CodeT5-test-12-300-4-2023-09-26-v3/results/checkpoint-78656'
# model_name = '../5-domain-pre-training/results/checkpoint-12290'
model_name = "CodeT5"
# model_name = '../4-qode2desc/no_bootstrap/analysis_report_CodeTrans_domain-checkpoint-12290-test-10-512-16-2023-10-07/results/checkpoint-2500'
# model_name = '../5-domain-pre-training/arxiv_pretraining/results/checkpoint-12290'

# model_name = "../5-domain-pre-training/analysis_report_CodeTrans-12-300-15-4-202310145_1/results/checkpoint-58500"
#model_name = "../5-domain-pre-training/analysis_report_CodeT5-4-300-15-4-202310145_1/results/checkpoint-19500"


DATE = "20231017"

SAMPLE = "test"
if SAMPLE == "test":
    load_best_model_at_end = False
else:
    load_best_model_at_end = None

# tokenization
encoder_max_length = 512
decoder_max_length = 150
RS = 42
LR = 5e-5

EPOCHS = 10
TRAIN_BATCH = 16
EVAL_BATCH = 4

WARMUP_STEPS = 500
WEIGHT_DECAY = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1
SAVE_STRATEGY = "steps"

LABEL_SMOOTHING = 0.1
PREDICT_GENERATE = True

MODE = "no_context"

EVAL_COLUMNS = [
    "eval_loss",
    "eval_rouge1",
    "eval_rouge2",
    "eval_rougeL",
    "eval_rougeLsum",
    "eval_bleu",
    "eval_gen_len",
]


analysis_name = (
    model_name
    + "-"
    + MODE
    + "-"
    + SAMPLE
    + "-"
    + str(EPOCHS)
    + "-"
    + str(encoder_max_length)
    + "-"
    + str(TRAIN_BATCH)
)

if len(model_name) > 15:
    analysis_name = (
        model_name.split("/")[-1]
        + "-"
        + SAMPLE
        + "-"
        + str(EPOCHS)
        + "-"
        + str(encoder_max_length)
        + "-"
        + str(TRAIN_BATCH)
    )

analysis_name = "CodeT5_domain_4_19500"
analysis_name = analysis_name + "-" + DATE + "_" + str(RS)

print(analysis_name)

CodeT5_domain_4_19500-20231017_42


In [7]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [8]:
print(analysis_name)
analysis_modules.scs_analyze(
    analysis_name=analysis_name,
    model_name=model_name,
    train_data_path=f"../../data/preprocessed/Quantlet/{DATE}/{MODE}/",
    train_data_name=f"full_train_dataset_{DATE}_sample0.json",
    val_data_path=f"../../data/preprocessed/Quantlet/{DATE}/{MODE}/",
    val_data_name=f"test_dataset_{DATE}_sample0.json",
    encoder_max_length=encoder_max_length,
    decoder_max_length=decoder_max_length,
    random_state=RS,
    eval_columns_list=EVAL_COLUMNS,
    learning_rate=LR,
    epochs=EPOCHS,
    train_batch=TRAIN_BATCH,
    eval_batch=EVAL_BATCH,
    warmup_steps=WARMUP_STEPS,
    weight_decay=WEIGHT_DECAY,
    logging_stes=LOGGING_STEPS,
    save_total_lim=SAVE_TOTAL_LIM,
    save_strategy=SAVE_STRATEGY,
    label_smooting=LABEL_SMOOTHING,
    predict_generate=PREDICT_GENERATE,
    load_best_model_at_end=load_best_model_at_end,
)

CodeT5_domain_4_19500-20231017_42
CodeT5_domain_4_19500-20231017_42
cuda


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1322: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


cuda


Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-5c480fa98444650e/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Step,Training Loss
100,7.982900
200,4.939100
300,4.501300
400,4.139000
500,3.757900
600,3.471800
700,3.247200
800,3.071400
900,2.908600
1000,2.788800


/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      3.865        0.324        0.162        0.286           0.291   

   eval_bleu  eval_gen_len  
0      0.056        18.218  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


__________
Original: Algorithm that identify the S&P500 firm tickers from news text. The algorithm only identify those tickers in the three most commonly seen cases. Detailed description refer to the paper
Summary before Tuning: Symbols Extraction in Spx Tickers
Summary after Tuning: This Quantlet is dedicated to gathering data on the input data from the SPX
__________


__________
Original: Inspect the derived variables from the News Network, Plots and Stats
Summary before Tuning: Learning Attention Variable for SpX Artificial Networks
Summary after Tuning: This quantlet includes two main code files. The first one named SPX_lead.
__________


__________
Original: Inspect the derived variables from the News Network, Plots and Stats
Summary before Tuning: PlotKernelDensityEstimator: A new object for plotting a series of
Summary after Tuning: Plotting Kernel Density Estimation for Epanonnikov Data
__________


__________
Original: Estimates and plots (yearly) empirical pricing kernels (E

In [9]:
gc.collect()
torch.cuda.empty_cache()